# Predicción de Promociones de Empleados

![Empleados](https://www.groupe.io/wp-content/uploads/2019/04/Best-Employee.jpg)

## En este projecto vamos a visualiar y analizar los datos referidos a empleados de una organización masiva, para luego poder entrenar un algoritmo de clasificación que pueda predecir cuál empleado será promovido

In [ ]:
import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='darkgrid')
sns.set_palette("pastel")
import plotly.express as px
from IPython.display import Image
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df = pd.read_csv("/kaggle/input/hr-analytics-classification/train_LZdllcl.csv")
test_full = pd.read_csv("/kaggle/input/hr-analytics-classification/test_2umaH9m.csv")
df.head()

In [ ]:
len(df)

In [ ]:
df.info()

Podemos ver que hay algunos valores Nulos para las variables de educación y rating del año anterior. En estas situaciones, tenemos dos caminos: eliminar las filas enteras que contienen datos inválidos, o imputarles algún valor. En este caso en particular, ya que solo vamos a utilizar estos datos para entrenar nuestro algoritmo, vamos a proceder a eliminar las filas.

In [ ]:
df = df.dropna()
len(df)

Al observar cuidadosamente, vemos que una variable numérica es de tipo "float64", lo que nos puede dificultar hacer algunas operaciones en el futuro, así que vamos a cambiar su formato a "integer".

In [ ]:
df['previous_year_rating'] = df['previous_year_rating'].astype(np.int64)

## Análisis de Datos y Visualización

In [ ]:
promo = pd.DataFrame(df.is_promoted.value_counts())
promo.columns = ["Promoted"]
promo["Valores"] = promo.index
promo["Valores"] = promo["Valores"].map({0:"No Promovido", 1:"Promovido"})
fig = px.pie(promo, values= "Promoted", names = "Valores", title = "Empleados Promovidos", color_discrete_sequence=px.colors.qualitative.Pastel2)
fig.update_layout(
    autosize=False,
    width=1000,
    height=500)
fig.show()

Naturalmente, la variable está muy desbalanceada: tenemos pocos casos de promociones comparados con los empleados que no son promovidos, como es de esperar en cualquier organización.

## Departamentos

In [ ]:
depart = pd.DataFrame(df.department.value_counts())
depart.columns = ["Valores"]
depart["Departamentos"] = depart.index
fig = px.bar(depart, x = "Departamentos", y= "Valores", title = "Cantidad de Empleados por Departamento", color_discrete_sequence=px.colors.qualitative.Pastel2)
fig.update_layout(
    autosize=False,
    width=850,
    height=650)
fig.show()

In [ ]:
depart = pd.DataFrame(df.groupby("is_promoted").department.value_counts().reset_index(name='Valores'))
for i in range(0,9):
    value = round(depart[depart["is_promoted"] == 1].iloc[i,2] / depart[depart["is_promoted"] == 0].iloc[i,2] * 100, 2)
    nombre = depart.iloc[i,1]
    print(f"Porcentaje de Empleados Promovidos en Departamento "+ str(nombre) + ": " + str(value) + "%")
depart["is_promoted"] = depart["is_promoted"].map({0:"No Promovido", 1:"Promovido"})

plt.figure(figsize=(12, 8))
sns.barplot(x='department', y='Valores', data=depart, hue='is_promoted')
plt.legend(title='Promoción', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xlabel('Empleados Promovidos y no Promovidos por Departamento')
plt.show()

Observamos que las promociones son relativas a la cantidad de empleados de cada departamento. A simple vista no hay ningún departamento que llame la atención. Quizás promover un número equitativo de empleados por departamento incluso sea una política de la empresa. Conocer esta informacións nos sería útil para nuestra tarea.

## Región 

In [ ]:
region = pd.DataFrame(df.region.value_counts())
region.columns = ["Regiones"]
region["Valores"] = region.index
fig = px.pie(region, values= "Regiones", names = "Valores", title = "Cantidad de Empleados por Regiones",color_discrete_sequence=px.colors.qualitative.Set2)
fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
fig.update_layout(
    autosize=False,
    width=1000,
    height=500)
fig.show()
print("Los empleados de la organización provienen de " + str(df.region.nunique()) + " regiones diferentes.")

## Educación

In [ ]:
educ = pd.DataFrame(df.groupby("is_promoted").education.value_counts().reset_index(name='Valores'))
for i in range(0,3):
    value = round(educ[educ["is_promoted"] == 1].iloc[i,2] / educ[educ["is_promoted"] == 0].iloc[i,2] * 100, 2)
    nombre = educ.iloc[i,1]
    print(f"Porcentaje de Empleados Promovidos en Departamento "+ str(nombre) + ": " + str(value) + "%")

educ["is_promoted"] = educ["is_promoted"].map({0:"No Promovido", 1:"Promovido"})

plt.figure(figsize=(12, 8))
sns.barplot(x='education', y='Valores', data=educ, hue='is_promoted')
plt.legend(title='Promoción', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xlabel('Empleados Promovidos y no Promovidos por Departamento')
plt.show()

Se puede observar que la cantidad de empleados promovidos en relación a la educación alcanzada es relativa a la cantidad absoluta de los empleados en cada una de estas categorías.

## Género

In [ ]:
genero = pd.DataFrame(df.groupby("is_promoted").gender.value_counts().reset_index(name='Valores'))

plt.figure(figsize=(13, 8))
sns.barplot(x='gender', y='Valores', data=genero, hue='is_promoted')
plt.title("Empleados Promovidos y no Promovidos por Género")
plt.legend(title='Promovido', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xlabel('Género')
plt.show()

## Canal de Reclutamiento

In [ ]:
canal = pd.DataFrame(df.groupby("is_promoted").recruitment_channel.value_counts().reset_index(name='Valores'))

plt.figure(figsize=(12, 8))
sns.barplot(x='recruitment_channel', y='Valores', data=canal, hue='is_promoted')
plt.title("Empleados Promovidos y no Promovidos por Canal de Reclutamiento")
plt.legend(title='Promovido', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xlabel('Canal de Reclutamiento')
plt.show()

## Cantidad de Entrenamientos

In [ ]:
entrenamientos = pd.DataFrame(df.groupby("is_promoted").no_of_trainings.value_counts().reset_index(name='Valores'))
promo_entrenamientos = entrenamientos[entrenamientos["is_promoted"]==1]
plt.figure(figsize=(12, 8))
sns.barplot(data=promo_entrenamientos, x="no_of_trainings", y="Valores")
plt.title("Empleados Promovidos por Cantidad de Capacitaciones Previos")
plt.xlabel('Cantidad de Capacitaciones')

Observamos que la mayoría de los empleados promovidos solo ha tenido 1 capacitación. El mayor número de capacitaciones ha sido 6 para los empleados promovidos. La cantidad de capacitaciones no parece ser un determinante muy fuerte para la promoción en esta organización.

## Variables Numéricas

In [ ]:
numericas = ["age", "previous_year_rating", "length_of_service","KPIs_met >80%","awards_won?","avg_training_score"]
df[numericas].hist(figsize=(16,8),layout=(2,3), density=True)
plt.show()

## Edad

In [ ]:
fig = px.histogram(df, x="age", title = "Distribución de Edades de los Empleados", color = "is_promoted", labels= {"age":"Edad","is_promoted": "Promoción"},nbins=35, color_discrete_sequence=px.colors.qualitative.Pastel2)
fig.update_layout(yaxis_title=" ")
fig.update_layout(
    autosize=False,
    width=900,
    height=650)
fig.show()
print("La media de edad de los empleados es " + str(round(np.mean(df.age),0)))
print("Le mediana de edad de los empleados es " + str(np.median(df.age)))

Como es de esperar, la distribución de edad de los empleados es similar a la de la población activa. Además de los estadísticos descriptivos, a simple vista podemos ver que la mayoría de los empleados tienen entre 26 y 43 años y que la distribución de edades de los empleados promovidos es relativa a la distribución total de edades de empleados.

In [ ]:
fig = px.histogram(df, x="previous_year_rating", title = "Distribución de Puntajes del Año Previo de los Empleados", color = "is_promoted", labels= {"previous_year_rating":"Puntajes","is_promoted": "Promoción"},nbins=35, color_discrete_sequence=px.colors.qualitative.Pastel2)
fig.update_layout(yaxis_title=" ")
fig.update_layout(
    autosize=False,
    width=900,
    height=650)
fig.show()
print("La media de edad de los empleados no promovidos es " + str(round(np.mean(df[df["is_promoted"]==0].previous_year_rating),2)))
print("La media de edad de los empleados promovidos es " + str(round(np.mean(df[df["is_promoted"]==1].previous_year_rating),2)))

En esta variable podemos empezar a ver una diferencia entre los empleados promovidos y los no promovidos. Probablemente esta variable, como las otras basadas en el rendimiento sí sean buenos predictores para conocer quién será promovido.

In [ ]:
fig = px.histogram(df, x="length_of_service", title = "Distribución de Cantidad de años de Servicio de los Empleados", color = "is_promoted", labels= {"length_of_service":"Años de Servicio","is_promoted": "Promoción"},nbins=35, color_discrete_sequence=px.colors.qualitative.Pastel2)
fig.update_layout(yaxis_title=" ")
fig.update_layout(
    autosize=False,
    width=900,
    height=650)
fig.show()
print("La media de años de servicio de los empleados no promovidos es " + str(round(np.mean(df[df["is_promoted"]==0].length_of_service),2)))
print("La media de años de servicio de los empleados promovidos es " + str(round(np.mean(df[df["is_promoted"]==1].length_of_service),2)))

In [ ]:
fig = px.histogram(df, x="KPIs_met >80%", title = "Distribución de Cantidad de Empleados que superaron el 80 de sus KPI", color = "is_promoted", labels= {"KPIs_met >80%":"KPI > 80","is_promoted": "Promoción"},nbins=35, color_discrete_sequence=px.colors.qualitative.Pastel2)
fig.update_layout(yaxis_title=" ")
fig.update_layout(
    autosize=False,
    width=700,
    height=650)
fig.show()

kpi_no = round(len(df[(df["is_promoted"]==1) & (df["KPIs_met >80%"] == 0)]) / len(df[df["KPIs_met >80%"] == 0]) *100, 2)
kpi_si = round(len(df[(df["is_promoted"]==1) & (df["KPIs_met >80%"] == 1)]) / len(df[df["KPIs_met >80%"] == 1]) *100, 2)

print("El " + str(kpi_no) + "% de empleados que no superó el 80% de sus KPI fue promovido")
print("El " + str(kpi_si) + "% de empleados que sí superó el 80% de sus KPI fue promovido")

Como suponíamos, esta variable de rendimiento también se muestra como un criterio para la promoción de empleados.

In [ ]:
fig = px.histogram(df, x="awards_won?", title = "Distribución de Cantidad de Empleados por Premios Obtenidos", color = "is_promoted", labels= {"awards_won?":"Premios Obtenidos","is_promoted": "Promoción"},nbins=35, color_discrete_sequence=px.colors.qualitative.Pastel2)
fig.update_layout(yaxis_title=" ")
fig.update_layout(
    autosize=False,
    width=700,
    height=650)
fig.show()

kpi_no = round(len(df[(df["is_promoted"]==1) & (df["awards_won?"] == 0)]) / len(df[df["awards_won?"] == 0]) *100, 2)
kpi_si = round(len(df[(df["is_promoted"]==1) & (df["awards_won?"] == 1)]) / len(df[df["awards_won?"] == 1]) * 100, 2)

print("El " + str(kpi_no) + "% de empleados que no obtuvo premios fue promovido")
print("El " + str(kpi_si) + "% de empleados que sí obtuvo permios fue promovido")

In [ ]:
fig = px.histogram(df, x="avg_training_score", title = "Distribución de Cantidad de años de Servicio de los Empleados", color = "is_promoted", labels= {"avg_training_score":"Promedio de Puntaje de Capacitación","is_promoted": "Promoción"},nbins=35, color_discrete_sequence=px.colors.qualitative.Pastel2)
fig.update_layout(yaxis_title=" ")
fig.update_layout(
    autosize=False,
    width=900,
    height=650)
fig.show()
print("La media de años de servicio de los empleados no promovidos es " + str(round(np.mean(df[df["is_promoted"]==0].avg_training_score),2)))
print("La media de años de servicio de los empleados promovidos es " + str(round(np.mean(df[df["is_promoted"]==1].avg_training_score),2)))

También observamos que esta variable tiene bastante peso para la promoción de empleados. Además de la diferencia en la media, podemos ver que son pocos empleados promovidos en los puntajes más bajos (alrededor del 50%) si lo ponemos en relación a la cantidad de empleados totales que obtuvieron ese puntaje promedio. Sin embargo, para los puntajes más altos (superiores al 90%), casi todos los empleados que obtienen estos puntajes son promovidos.

## Modelo de Predicción

### Limpieza de datos para entrenar el modelo

En primer lugar, vamos a eliminar la fila de ID de empleados porque no nos es útil para el modelo predictivo y las regiones por la gran cantidad que tenemos, lo que puede llevar a menor precisión

In [ ]:
df.drop(["employee_id", "region"], axis=1, inplace=True)

Ahora necesitamos convertir las variables categóricas en numéricas, para esto vamos a convertir cada una de las categorías de estas variables en valores numerales como 0-1-2, etc. De acuerdo a la cantidad de categorías que tenga cada variable.

In [ ]:
df.department = df.department.map({"Sales & Marketing":0,
                 "Operations":1,
                 "Technology":2,
                 "Analytics":3,
                 "R&D":4,
                 "Procurement":5,
                 "Finance":6,
                 "HR":7,
                 "Legal":8})
df.education = df.education.map({"Master's & above":0,
                                "Bachelor's":1,
                                "Below Secondary":2})
df.gender = df.gender.map({"f":0,
                          "m":1})
df.recruitment_channel = df.recruitment_channel.map({"sourcing":0,
                                                    "other":1,
                                                    "referred":2})
df.head()

Ahora vamos a entrenar y comparar los puntajes de múltiples algoritmos de clasificación. Decidí ir por los más comunes, que son: Regresión Logística, Máquinas de Vector Soporte, Árboles de Decisión, Bosques Aleatorios de Clasificación, Clasificador Bayesiano Ingenuo y Potenciación Extrema de Gradiente (Extreme Gradient Boosting)

Como base, utilicé el [código de Roberto Salazar](http://https://towardsdatascience.com/machine-learning-classifiers-comparison-with-python-33149aecdbca) para esta tarea

In [ ]:
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import cross_validate, StratifiedKFold, RandomizedSearchCV

scoring = {'accuracy':make_scorer(accuracy_score), 
           'precision':make_scorer(precision_score),
           'recall':make_scorer(recall_score), 
           'f1_score':make_scorer(f1_score),
            'roc_auc':make_scorer(roc_auc_score)}

log_model = LogisticRegression(max_iter=10000)
svc_model = LinearSVC(dual=False)
dtr_model = DecisionTreeClassifier()
rfc_model = RandomForestClassifier()
gnb_model = GaussianNB()
xgb_model = xgb.XGBClassifier()

def models_evaluation(X, y, folds):
    log = cross_validate(log_model, X, y, cv=folds, scoring=scoring)
    svc = cross_validate(svc_model, X, y, cv=folds, scoring=scoring)
    dtr = cross_validate(dtr_model, X, y, cv=folds, scoring=scoring)
    rfc = cross_validate(rfc_model, X, y, cv=folds, scoring=scoring)
    gnb = cross_validate(gnb_model, X, y, cv=folds, scoring=scoring)
    xgb = cross_validate(xgb_model, X, y, cv=folds, scoring=scoring)


    models_scores_table = pd.DataFrame({'Logistic Regression':[log['test_accuracy'].mean(),
                                                               log['test_precision'].mean(),
                                                               log['test_recall'].mean(),
                                                               log['test_f1_score'].mean(),
                                                              log['test_roc_auc'].mean()],
                                       
                                      'Support Vector Classifier':[svc['test_accuracy'].mean(),
                                                                   svc['test_precision'].mean(),
                                                                   svc['test_recall'].mean(),
                                                                   svc['test_f1_score'].mean(),
                                                                  svc['test_roc_auc'].mean()],
                                       
                                      'Decision Tree':[dtr['test_accuracy'].mean(),
                                                       dtr['test_precision'].mean(),
                                                       dtr['test_recall'].mean(),
                                                       dtr['test_f1_score'].mean(),
                                                      dtr['test_roc_auc'].mean()],
                                       
                                      'Random Forest':[rfc['test_accuracy'].mean(),
                                                       rfc['test_precision'].mean(),
                                                       rfc['test_recall'].mean(),
                                                       rfc['test_f1_score'].mean(),
                                                      rfc['test_roc_auc'].mean()],
                                       
                                      'Gaussian Naive Bayes':[gnb['test_accuracy'].mean(),
                                                              gnb['test_precision'].mean(),
                                                              gnb['test_recall'].mean(),
                                                              gnb['test_f1_score'].mean(),
                                                             gnb['test_roc_auc'].mean()],
                                        
                                       'Extreme Gradient Boosting':[xgb['test_accuracy'].mean(),
                                                              xgb['test_precision'].mean(),
                                                              xgb['test_recall'].mean(),
                                                              xgb['test_f1_score'].mean(),
                                                            xgb['test_roc_auc'].mean()]},
                                      
                                      index=['Accuracy', 'Precision', 'Recall', 'F1 Score', "AUC ROC"])
    

    models_scores_table['Best Score'] = models_scores_table.idxmax(axis=1)

    return(models_scores_table)
  
models_evaluation(features, target, 5)

Al interpretar estos resultados, debemos tener en cuenta lo siguiente:
1. El desbalance de nuestra variable objetivo: Como vimos al principio, la cantidad de empleados no promovidos es mayor al 90%. Esto significa que, si un modelo predictivo diera como 100% de toda la muestra como "no promovido", estaría acertando en el 90% de los casos. Por esta razón, el puntaje de "Accuracy" (Exactitud) de un modelo predictivo en esta situación no es el mejor a utilizar.
2. El problema que buscamos resolver: la principal demanda de la organización es la de detectar con anticipación aquellos empleados que pueden llegar a ser promovidos para poder prepararlos con mayor tiempo. Haremos la suposición de que tanto la tasa de "Precisión" como la de "Recuperación" tienen el mismo costo para la empresa, y por esta razón utilizaremos el puntaje F1 que es un equilibrio de ambos puntajes. El algoritmo que mejor se desempeña sobre este puntaje es el Extreme Gradient Boosting.

## Ajuste de hiperparámetros

Debido a que este projecto es una simple demostración, no vamos a realizar una optimización de hiperparámetros a fuerza bruta porque llevaría mucho tiempo. Decidí realizar una optimización mediante una búsqueda aleatoria de hiperparámetros.

In [ ]:
from datetime import datetime

def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

        
params = {
        'min_child_weight': [1,3, 5, 7, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.2, 0.4, 0.6, 0.8, 1.0],
        'colsample_bytree': [0.2, 0.4, 0.6, 0.8, 1.0],
        'max_depth': [2, 3, 4, 5, 7,9]
        }

folds = 5
param_comb = 20

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 7)

random_search = RandomizedSearchCV(xgb_model, param_distributions=params, n_iter=param_comb, scoring='f1',verbose = 3, n_jobs=4, cv=skf.split(features,target), random_state=7)


start_time = timer(None) 
random_search.fit(features, target)
timer(start_time)


In [ ]:
print('Parámetros Óptimos:')
print(random_search.best_params_)
print("\n Mejor Puntaje F1 Obtenido")
print(random_search.best_score_)

## Predicciones

Ahora que tenemos el algoritmo de clasificación optimizado, podemos utilizarlo para realizar predicciones sobre nuevos datos y saber cuál empleado será promovido con anticipación, siempre teniendo en cuenta la falibilidad del modelo y el puntaje obtenido en la prueba.

1. Al leer la nueva base de datos, debemos recordar que tenemos que hacer la misma limpieza de datos que hicimos para entrenar al algoritmo, es decir, eliminar las columnas de "ID" de los empleados y las regiones, además de imputar los mismos valores numéricos a las variables categóricas que utilizamos.

In [ ]:
test = test_full.copy() 

test.drop(["employee_id","region"],axis=1, inplace=True)

test.department = test.department.map({"Sales & Marketing":0,
                 "Operations":1,
                 "Technology":2,
                 "Analytics":3,
                 "R&D":4,
                 "Procurement":5,
                 "Finance":6,
                 "HR":7,
                 "Legal":8})
test.education = test.education.map({"Master's & above":0,
                                "Bachelor's":1,
                                "Below Secondary":2})
test.gender = test.gender.map({"f":0,
                          "m":1})
test.recruitment_channel = test.recruitment_channel.map({"sourcing":0,
                                                    "other":1,
                                                    "referred":2})

test.head()

El resultado final, es una base de datos que mostrará cuál empleado será promovido (1) y cuál no (0), y la probabilidad de que estos empleados tengan de ser promovidos de acuerdo a este modelo.

In [ ]:
y_test = random_search.predict(test)
y_test_prob = random_search.predict_proba(test)
resultados = pd.DataFrame(data={'id':test_full['employee_id'], 'Promovido':y_test, "Probabilidad":y_test_prob[:,1]})
print(resultados.head())
print("La cantidad de Empleados Promovidos es de: " + str(len(resultados[resultados["Promovido"]==1])))